In [1]:
import pandas as pd
import numpy as np
# from IPython.display import display
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import FeatureUnion
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error

In [2]:
usecols = ['AGE', 'ANL_INCM', 'ANVRSRY_DATE','BIRTH_DATE','COMNCTN_IND', 'Enquiry Customer Type', 'Enquiry Source',
 'Enquiry Date', 'Enquiry Type','link_Enquiry', 'link_Model', 'LOCTN_CD', 'MARTL_IND', 'MOTHER_TONGE', 'No Of Followup',
 'Occupation Desc', 'PARNT_GROP', 'PERMNT_PIN', 'Procrurement Offer Responce', 'RELGN_CD', 'RES_PIN', 'SEX_IND',
 'SORC_CD', 'Web Enquiry Flag', 'LEAD_TYPE']
parse_dates = ['ANVRSRY_DATE', 'Enquiry Date']
dtype_dict = {'AGE': 'Int8', 'ANL_INCM': 'category', 'ANVRSRY_DATE': 'str','BIRTH_DATE':'str', 'COMNCTN_IND': 'category', 'CORPRT_CD': 'category',
 'Enquiry Customer Type': 'category', 'Enquiry Source': 'category', 'Enquiry Date': 'str', 'Enquiry Type': 'category','link_Enquiry':'float64',
 'link_Model': 'object', 'LOCTN_CD': 'object', 'MARTL_IND': 'category', 'MOTHER_TONGE': 'Int8', 'No Of Followup': 'Int16',
 'Occupation Desc': 'category', 'PARNT_GROP': 'object', 'PERMNT_PIN': 'Int32', 'Procrurement Offer Responce': 'category',
 'RELGN_CD': 'category', 'RES_PIN': 'Int32', 'SEX_IND': 'category', 'SORC_CD': 'category', 'Web Enquiry Flag': 'category',
 'LEAD_TYPE': 'category'}

In [8]:

def load_Enq():
    enq = pd.read_csv("Enquiry new data.csv",
                  usecols= usecols,
#                   dtype=dtype_dict,
                  parse_dates=parse_dates,
                  nrows=100000,
                  error_bad_lines=False,
#                   na_filter=False,
                  engine='c',
                  low_memory=False)

#     csv_path=pd.read_csv("Enquiry new data.csv", encoding="Cp1252", low_memory=False,nrows=100000)
    return enq

In [4]:
def load_retail():
    csv_path=pd.read_csv("retail.csv", encoding="Cp1252", low_memory=False)
    return csv_path

In [5]:
retail=load_retail()
retail['Enquiry Date']=pd.to_datetime(retail['Enquiry Date'])


In [9]:
enq=load_Enq()

In [ ]:
enq.isnull().sum()

In [ ]:
enq.isnull().sum()

In [10]:
retail.rename(columns= {'Invoice Status':'Invoice_Status'}, inplace=True)

In [11]:
#to check if enquiry is present in retail table (Y lable)
def checkInRetail(row):
    if row["link_Enquiry_enq"]==row["link_Enquiry"]:
        return 1
    else:
        return 0
    

In [12]:
enq["ANVRSRY_DATE"]=pd.to_datetime(enq["ANVRSRY_DATE"], errors = 'coerce')
enq["Enquiry Date"]=pd.to_datetime(enq["Enquiry Date"])
enq["BIRTH_DATE"]=pd.to_datetime(enq["BIRTH_DATE"])

In [13]:
# to check if enquiry was made near birth/anniversary date
def anniversaryVsBirthVsEnquiryDate(row):
    
#(enq["BIRTH_DATE"]-enq["Enquiry Date"]).astype('<m8[M]')
   return (1 if (abs(row["BIRTH_DATE"].month - row["Enquiry Date"].month)<=1 or \
           abs(row["ANVRSRY_DATE"].month - row["Enquiry Date"].month)<=1) else 0)
        
        
enq["celebration"]=enq.apply(anniversaryVsBirthVsEnquiryDate,axis=1)



In [14]:
enq.dropna(subset=['ANL_INCM'],inplace=True)

In [ ]:
null_rate = enq.isnull().sum(axis = 0).sort_values(ascending = False)/float((len(enq)))
# df_.dropna(axis = 0,thresh=30,inplace = True)

In [ ]:
enq.drop(null_rate[null_rate>0.7].index,axis = 1,inplace=True)

In [ ]:
enq.dropna(axis = 0,thresh=30,inplace = True)

In [ ]:
unique_rate = enq.apply(lambda x: len(pd.unique(x)),axis = 0).sort_values(ascending = False)

In [ ]:
enq["BIRTH_DATE"].dt.month - enq["Enquiry Date"].dt.month

In [ ]:
# retail.dropna(subset=['link_Enquiry'],inplace=True)

In [15]:
enq.rename(columns={"link_Enquiry":"link_Enquiry_enq"}, inplace=True)
# retail=retail.query("Invoice_Type=='CI' and Invoice_Status=='Invoiced'")
enq=pd.merge(enq,retail.link_Enquiry.to_frame(),left_on=["link_Enquiry_enq"], right_on=["link_Enquiry"],how="left")


In [16]:
## Check in enquiry data in retail
enq["result"]=enq.apply(checkInRetail,axis=1)

In [ ]:
enq.info()

In [17]:
enq.drop('link_Enquiry',inplace=True,axis=1)
enq.drop_duplicates(inplace=True)

## check null values by the result values

In [ ]:
print(enq[enq['result']==0].isnull().sum())
enq[enq['result']==0].isnull().sum().plot()

In [ ]:
print(enq[enq['result']==1].isnull().sum())
enq[enq['result']==1].isnull().sum().plot()

In [ ]:
# enq[enq['result']==0].isnull().sum()

## By the above graph the result '0' values having more number of null values than '1', then applied 'dropna' upon the '0' values

In [18]:
enq1=enq[enq['result']==0].dropna()

In [19]:
enq2=pd.concat([enq1,enq[enq['result']==1]],axis=0)

In [20]:
# enq.drop_duplicates(inplace=True)

In [21]:
retail['Enquiry Date']=pd.to_datetime(retail['Enquiry Date'])

In [22]:
retail['Enquiry Date'].dt.year.value_counts()

2018    204419
2019     88281
2017      5709
2016        18
2015         3
Name: Enquiry Date, dtype: int64

In [23]:
retail['Enquiry Date'].dt.year.value_counts()

2018    204419
2019     88281
2017      5709
2016        18
2015         3
Name: Enquiry Date, dtype: int64

In [ ]:
enq['result'].value_counts().plot(kind='bar')

In [24]:
enq=enq2
enq["models"]=enq["link_Model"].str[:4]
# enq["result"].value_counts()

In [25]:
# to impute null fields of AGE column
mean=enq["AGE"].mean()
std=enq["AGE"].std()

is_null = (~enq["AGE"].notnull()).sum()
# compute random numbers between the mean, std and is_null
rand_age=np.random.randint(mean-std,mean+std,size=is_null)
age_slice=enq["AGE"].copy()
age_slice[~enq.AGE.notnull()]=rand_age
# print(rand_age)
# break
enq["AGE"]=age_slice

In [26]:
# enq['SORC_CD'].value_counts()

In [27]:
# finalEnq[finalEnq['result']==1]['MARTL_IND'].value_counts(normalize=True)

In [28]:
# enq[enq['result']==1]['ANL_INCM'].value_counts()

In [29]:
finalEnq=enq.drop(columns=['ANVRSRY_DATE','MOTHER_TONGE','RELGN_CD','BIRTH_DATE','link_Model','PARNT_GROP','PERMNT_PIN','RES_PIN','link_Enquiry_enq',"Procrurement Offer Responce","Web Enquiry Flag",'LEAD_TYPE'],axis=1)

In [30]:
names=['0-10','10-20','20-30','30-40','40-50','50-60','60-70','70-80','80-90','90>']
bins=[0,10,20,30,40,50,60,70,80,90]
d = dict(enumerate(names, 1))

finalEnq['AgeRange'] = np.vectorize(d.get)(np.digitize(finalEnq['AGE'], bins))

In [31]:
finalEnq["AgeRange"]=finalEnq["AgeRange"].astype('category')
finalEnq.drop('AGE',axis=1,inplace=True)

In [32]:
finalEnq['month']=finalEnq['Enquiry Date'].dt.month
finalEnq['day_of_week']=finalEnq['Enquiry Date'].dt.weekday_name

In [33]:
finalEnq.drop('Enquiry Date',inplace=True,axis=1)

In [34]:
enq[enq['result']==0].shape

(23169, 28)

In [35]:
# Function to draw conditional distribution plot splot by whether the customer subscribed
def draw_conditional_distplot(feature,df):
    # Seat seaborn to use nice colors
#     sns.set(palette=five_thirty_eight)
    # Draw the no plot
    sns.distplot(df[df.result == 0][feature],label='no')
    # Draw the yes plot
    sns.distplot(df[df.result == 1][feature],label='yes')
    # Draw the legend
    plt.legend()
    # Display the plot
    plt.show()

In [ ]:
finalEnq['Enquiry Source'][0]

In [ ]:
finalEnq[finalEnq.result == 0]['Enquiry Source'].value_counts().plot(kind='bar')

In [ ]:
# finalEnq["Enquiry Source"]=finalEnq["Enquiry Source"].astype('str')

# draw_conditional_distplot('Enquiry Source',finalEnq)

In [36]:
draw_conditional_distplot(feature,df)

NameError: name 'feature' is not defined

In [ ]:
# finalEnq['day_of_week']=finalEnq['BIRTH_DATE'].dt.weekday_name.value_counts().plot(kind='bar')

In [ ]:
# # Create old people group
# finalEnq['age_old'] = np.where(finalEnq['AGE'] >= 60, 1,0)
# # Create mid age people group
# finalEnq['age_mid'] = np.where((finalEnq['AGE'] <= 60) & (finalEnq['AGE'] >= 35), 1,0)
# # Create young people group
# finalEnq['age_young'] = np.where(finalEnq['AGE'] <= 35, 1,0)

In [37]:
#removing Location cd having less than or equal to 5 occurences  in the data set
lc = pd.DataFrame(enq["LOCTN_CD"].value_counts()<=5)

# lc.reset_index(inplace=True)
filterLocn=lc[lc["LOCTN_CD"]==True].index

finalEnq.drop(index=finalEnq[finalEnq['LOCTN_CD'].isin(filterLocn)].index, axis=0,inplace=True)

In [38]:
#removing models having less than or equal to 5 occurences  in the data set
mod = pd.DataFrame(enq["models"].value_counts()<=5)
# lc.reset_index(inplace=True)
filterMod=mod[mod["models"]==True].index

finalEnq.drop(index=finalEnq[finalEnq['models'].isin(filterMod)].index, axis=0,inplace=True)

In [39]:
#removing Location cd having less than or equal to 5 occurences  in the data set
lc = pd.DataFrame(enq["Enquiry Source"].value_counts()<=5)

# lc.reset_index(inplace=True)
filterLocn=lc[lc["Enquiry Source"]==True].index

finalEnq.drop(index=finalEnq[finalEnq['Enquiry Source'].isin(filterLocn)].index, axis=0,inplace=True)

In [40]:
#removing Location cd having less than or equal to 5 occurences  in the data set
lc = pd.DataFrame(enq["No Of Followup"].value_counts()<=5)

# lc.reset_index(inplace=True)
filterLocn=lc[lc["No Of Followup"]==True].index

finalEnq.drop(index=finalEnq[finalEnq['No Of Followup'].isin(filterLocn)].index, axis=0,inplace=True)

In [41]:
#removing Location cd having less than or equal to 5 occurences  in the data set
lc = pd.DataFrame(enq["SORC_CD"].value_counts()<=5)

# lc.reset_index(inplace=True)
filterLocn=lc[lc["SORC_CD"]==True].index

finalEnq.drop(index=finalEnq[finalEnq['SORC_CD'].isin(filterLocn)].index, axis=0,inplace=True)

In [42]:
finalEnq["models"]=finalEnq["models"].astype('category')
# finalEnq["LOCTN_CD"]=finalEnq["LOCTN_CD"].astype('category')

In [43]:
finalEnq.drop("ANL_INCM",axis=1,inplace=True)

In [44]:
finalEnq.isnull().sum()

COMNCTN_IND                0
Enquiry Customer Type      0
Enquiry Source             0
Enquiry Type               0
LOCTN_CD                   0
MARTL_IND                838
No Of Followup             0
Occupation Desc          157
SEX_IND                  485
SORC_CD                    0
celebration                0
result                     0
models                     0
AgeRange                   0
month                      0
day_of_week                0
dtype: int64

In [45]:
# This function fills null values with highest value in the column
finalEnq = finalEnq.apply(lambda x: x.fillna(x.value_counts().index[0]))


In [46]:
# finalEnq[finalEnq['result']==0].shape

In [47]:
from sklearn import preprocessing
#Convert all the columns to string
finalEnq1=finalEnq.drop('result',axis=1)
input_x = finalEnq1.applymap(str)
# input_x_test=x_test.applymap(str)
input_x.dtypes.sort_index()

AgeRange                 object
COMNCTN_IND              object
Enquiry Customer Type    object
Enquiry Source           object
Enquiry Type             object
LOCTN_CD                 object
MARTL_IND                object
No Of Followup           object
Occupation Desc          object
SEX_IND                  object
SORC_CD                  object
celebration              object
day_of_week              object
models                   object
month                    object
dtype: object

In [66]:
finalEnq

,COMNCTN_IND,Enquiry Customer Type,Enquiry Source,Enquiry Type,LOCTN_CD,MARTL_IND,No Of Followup,Occupation Desc,SEX_IND,SORC_CD,celebration,result,models,AgeRange,month,day_of_week
95,RES,Individual,Newspaper,Walk-in,AR02,M,2,Business,M,NWP,1,0,JETO,20-30,6,Friday
99,RES,Individual,Newspaper,Walk-in,AR02,M,2,Agri-based,M,NWP,1,0,BOLP,20-30,5,Tuesday
104,RES,Corporate,DSA / DMA,Field,AR02,M,3,Business,M,DSA,0,0,MAXI,20-30,3,Friday
106,RES,Individual,Unplanned Activity,Field,MK01,M,3,Salaried-Others,M,UP,0,0,ALFP,20-30,8,Wednesday
110,RES,Individual,DSA / DMA,Field,AR02,M,2,Business,M,DSA,0,0,MAXI,20-30,1,Monday
111,PRM,Corporate,Newspaper,Field,AR02,M,4,Agri-based,M,NWP,1,0,BOLP,20-30,2,Wednesday
115,RES,Individual,Newspaper,Walk-in,AR02,M,3,Housewife,M,NWP,0,0,BOLP,20-30,1,Wednesday
116,PRM,Individual,Newspaper,Walk-in,MK02,M,2,Salaried-Others,M,NWP,1,0,PUPM,20-30,3,Monday
118,RES,Corporate,Newspaper,Walk-in,AR02,M,3,Business,M,NWP,0,0,BOLP,20-30,2,Monday
120,PRM,Individual,Newspaper,Field,AR02,M,3,Business,M,NWP,1,0,PUPM,20-30,1,Monday


In [48]:
input_x_array = np.array(finalEnq1)
# input_x_test_array=np.array(input_x_test)

for i in range(finalEnq1.shape[1]):
    lbl = preprocessing.LabelEncoder()
    lbl.fit(finalEnq1.iloc[:,i])
    finalEnq1.iloc[:, i] = lbl.transform(finalEnq1.iloc[:, i])

## Select K best

In [53]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2

In [54]:
best_feature=SelectKBest(score_func=chi2,k=10)
fit=best_feature.fit(finalEnq1,finalEnq['result'])

In [55]:
dfscores=pd.DataFrame(fit.scores_)
df_columns=pd.DataFrame(finalEnq1.columns)

In [56]:
featurescores=pd.concat([df_columns,dfscores],axis=1)

In [57]:
featurescores.columns=['specs','scores']

In [58]:
required_features=featurescores.nlargest(15,'scores')['specs'].values

In [59]:
input_x=finalEnq1[required_features]

In [60]:
required_features

array(['MARTL_IND', 'Occupation Desc', 'models', 'Enquiry Type', 'month',
       'SORC_CD', 'AgeRange', 'celebration', 'Enquiry Source', 'LOCTN_CD',
       'No Of Followup', 'day_of_week', 'Enquiry Customer Type',
       'SEX_IND', 'COMNCTN_IND'], dtype=object)

In [ ]:
# list(input_x_array[:,5])

In [ ]:
# for i in range(input_x_array.shape[1]):
#     lbl = preprocessing.LabelEncoder()
#     lbl.fit(list(input_x_array[:,i]))
#     input_x_array[:, i] = lbl.transform(input_x_array[:, i])

In [61]:
x_train, x_test, y_train, y_test = train_test_split(input_x, finalEnq['result'].values, test_size=0.3,random_state=42)

In [62]:
x_=pd.DataFrame(x_train)

In [63]:
## With class_weight='balanced'
from sklearn.ensemble import RandomForestClassifier

forest_clf = RandomForestClassifier(n_estimators=100, random_state=42,)
forest_scores = cross_val_score(forest_clf, np.array(input_x), finalEnq['result'], cv=3)
forest_scores.mean()

0.737843491745949

In [64]:
# With class_weight='balanced'
forest = RandomForestClassifier(n_estimators=100, random_state=42,class_weight='balanced')
forest.fit(x_train,y_train)

RandomForestClassifier(bootstrap=True, class_weight='balanced',
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, min_impurity_decrease=0.0,
                       min_impurity_split=None, min_samples_leaf=1,
                       min_samples_split=2, min_weight_fraction_leaf=0.0,
                       n_estimators=100, n_jobs=None, oob_score=False,
                       random_state=42, verbose=0, warm_start=False)

In [ ]:
y_pred_forest=forest.predict(x_test)
forest.score(x_test,y_test)

In [ ]:
from sklearn.metrics import accuracy_score

accuracy_score(y_test,y_pred_forest)

In [ ]:
y_pred_forest

In [ ]:
from sklearn.metrics import accuracy_score

accuracy_score(y_test,y_pred_forest)

In [ ]:
from sklearn.metrics import precision_score, recall_score

precision_score(y_test, y_pred_forest)

In [ ]:
recall_score(y_test, y_pred_forest)

In [ ]:
from sklearn.metrics import  classification_report
  
print(classification_report(y_test,y_pred_forest))

In [ ]:
finalEnq1.shape

In [ ]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, y_pred_forest)

In [ ]:
12325/(12325+3469)

In [ ]:
import pickle
with open('filter_random.pkl', 'wb') as t:
    pickle.dump(forest, t)

In [ ]:
output = open('Departure_encoder_filter.pkl', 'wb')
pickle.dump(lbl, output)
output.close()

In [ ]:
pkl_file = open('Departure_encoder_new_loct.pkl', 'rb')
le_departure = pickle.load(pkl_file) 
pkl_file.close()

In [ ]:
# with open("R_forest.pkl",'wb') as t:
#     pickle.dump(lbl,t,protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
pkl_file1 = open('R_forest_2_new_.pkl', 'rb')
model = pickle.load(pkl_file1) 
pkl_file1.close()

In [ ]:
pred_=model.predict(x_test)

In [ ]:
# lbl.inverse_transform(list(x_train[3]))

In [ ]:
# x_train.shape

In [ ]:
test_data=pd.read_csv("EnquiryToRetail Jun2019 to till date_new.csv")

In [ ]:
finalEnq['result'].value_counts().plot(kind='bar')

In [ ]:
enq.dropna()['result'].value_counts()

In [ ]:
text='AGE', 'ANL_INCM', 'ANVRSRY_DATE', 'BIRTH_DATE', 'COMNCTN_IND','CORPRT_CD', 'Enquiry Customer Type', 'Enquiry Date', 'Enquiry Source','Enquiry Type', 'link_Enquiry', 'link_Model', 'LOCTN_CD', 'MARTL_IND','MOTHER_TONGE', 'No Of Followup', 'Occupation Desc', 'PARNT_GROP','PERMNT_PIN', 'Procrurement Offer Responce', 'RELGN_CD', 'RES_PIN','SEX_IND', 'SORC_CD', 'Web Enquiry Flag', 'LEAD_TYPE'


In [ ]:
# p=[]
# for i in enq.columns.isin(text):
#     if i==True:
#         p.append(i)